In [124]:
! wget https://raw.githubusercontent.com/SchattenGenie/mlhep2019_2_phase/master/analysis/calogan_metrics.py
! wget https://raw.githubusercontent.com/SchattenGenie/mlhep2019_2_phase/master/analysis/prd_score.py
! wget https://raw.githubusercontent.com/SchattenGenie/mlhep2019_2_phase/master/analysis/score.py
! wget https://github.com/SchattenGenie/mlhep2019_2_phase/raw/master/analysis/embedder.tp
! wget https://github.com/SchattenGenie/mlhep2019_2_phase/raw/master/analysis/generator.py

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as utils
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook
import seaborn as sns
from IPython.display import clear_output
sns.set()

def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

device = 'cuda' if torch.cuda.is_available() else 'cpu'

--2020-02-11 16:35:43--  https://raw.githubusercontent.com/SchattenGenie/mlhep2019_2_phase/master/analysis/calogan_metrics.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4115 (4.0K) [text/plain]
Saving to: ‘calogan_metrics.py.1’

calogan_metrics.py. 100%[===================>]   4.02K  --.-KB/s    in 0s      

2020-02-11 16:35:43 (103 MB/s) - ‘calogan_metrics.py.1’ saved [4115/4115]

--2020-02-11 16:35:44--  https://raw.githubusercontent.com/SchattenGenie/mlhep2019_2_phase/master/analysis/prd_score.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200

In [125]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
train_data_path = '/gdrive/My Drive/mlhep2019_gan/data_train.npz'
val_data_path = '/gdrive/My Drive/mlhep2019_gan/data_val.npz'
test_data_path = '/gdrive/My Drive/mlhep2019_gan/data_test.npz'

In [127]:
N = 1000

data_train = np.load(train_data_path, allow_pickle=True)
print(list(data_train.keys()))

# [data_size, 900]
EnergyDeposit = data_train['EnergyDeposit'][:N]
# reshaping it as [data_size, channels, img_size_x, img_size_y]
# channels are needed for pytorch conv2d-layers
EnergyDeposit = EnergyDeposit.reshape(-1, 1, 30, 30)

# [data_size, 3]
ParticleMomentum = data_train['ParticleMomentum'][:N]

# [data_size, 2]
ParticlePoint = data_train['ParticlePoint'][:, :2][:N]

# [data_size, 1]
ParticlePDG = data_train['ParticlePDG'][:N]

['EnergyDeposit', 'ParticlePoint', 'ParticleMomentum', 'ParticlePDG']


In [0]:
EnergyDeposit = torch.tensor(EnergyDeposit).float()
ParticleMomentum = torch.tensor(ParticleMomentum).float()
ParticlePoint = torch.tensor(ParticlePoint).float()

BATCH_SIZE = 128
calo_dataset = utils.TensorDataset(EnergyDeposit, ParticleMomentum, ParticlePoint)
calo_dataloader = torch.utils.data.DataLoader(calo_dataset, batch_size=BATCH_SIZE, pin_memory=True, shuffle=True)

In [129]:
EnergyDeposit.shape

torch.Size([1000, 1, 30, 30])

In [168]:
def train(epoch):
    model.train()
    train_loss = 0
    for EnergyDeposit_b, ParticleMomentum_b, ParticlePoint_b in calo_dataloader:
            EnergyDeposit_b, ParticleMomentum_b, ParticlePoint_b = EnergyDeposit_b.to(device), \
                                                                   ParticleMomentum_b.to(device), \
                                                                   ParticlePoint_b.to(device)
            ParticleMomentum_ParticlePoint_b = torch.cat([ParticleMomentum_b.to(device), ParticlePoint_b.to(device)], dim=1)
            EnergyDeposit_b = EnergyDeposit_b.view(len(EnergyDeposit_b), -1)
            data = torch.cat([EnergyDeposit_b, ParticleMomentum_ParticlePoint_b], dim=1)
          #for batch_idx, (data, _) in enumerate(train_loader):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)
            loss = loss_function(recon_batch, data, mu, logvar)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(calo_dataloader.dataset)))

'''
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))
'''

"\ndef test(epoch):\n    model.eval()\n    test_loss = 0\n    with torch.no_grad():\n        for i, (data, _) in enumerate(test_loader):\n            data = data.to(device)\n            recon_batch, mu, logvar = model(data)\n            test_loss += loss_function(recon_batch, data, mu, logvar).item()\n\n    test_loss /= len(test_loader.dataset)\n    print('====> Test set loss: {:.4f}'.format(test_loss))\n"

In [0]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(905, 500)
        self.fc21 = nn.Linear(500, 30)
        self.fc22 = nn.Linear(500, 30)
        self.fc3 = nn.Linear(30, 500)
        self.fc4 = nn.Linear(500, 905)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 905))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    RMSE = torch.sqrt(torch.nn.functional.mse_loss(recon_x, x.view(-1, 905), reduction='sum'))

    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return RMSE + KLD

In [170]:
for epoch in range(1, 700):
        train(epoch)

====> Epoch: 1 Average loss: 54810298223160008.0000
====> Epoch: 2 Average loss: 2639.1718
====> Epoch: 3 Average loss: 537.3898
====> Epoch: 4 Average loss: 523.0493
====> Epoch: 5 Average loss: 551.4626
====> Epoch: 6 Average loss: 1099.7058
====> Epoch: 7 Average loss: 519.9940
====> Epoch: 8 Average loss: 501.7001
====> Epoch: 9 Average loss: 717.0695
====> Epoch: 10 Average loss: 500.1561
====> Epoch: 11 Average loss: 493.6598
====> Epoch: 12 Average loss: 42090.5484
====> Epoch: 13 Average loss: 482.6287
====> Epoch: 14 Average loss: 485.3702
====> Epoch: 15 Average loss: 480.1142
====> Epoch: 16 Average loss: 471.1477
====> Epoch: 17 Average loss: 462.7370
====> Epoch: 18 Average loss: 456.4029
====> Epoch: 19 Average loss: 448.1143
====> Epoch: 20 Average loss: 443.5032
====> Epoch: 21 Average loss: 438.3045
====> Epoch: 22 Average loss: 430.6312
====> Epoch: 23 Average loss: 424.3483
====> Epoch: 24 Average loss: 417.2792
====> Epoch: 25 Average loss: 414.3236
====> Epoch: 26 